### Algorithm for the grouping of the data from the Medibio's ECG file set. 

#### written by Nicola Pastorello 17/11/2015
* v.3.1 uses only profiles with reliable sleeping time extrapolation
* v.3.2 cleaned from useless parts

Loading libraries

In [5]:
library('party')
library('caret')
library('e1071')
library('RWeka')

Loading and cleaning data

In [122]:
originalDF = read.csv('metrics.csv')

# Check if missing data
is.na.data.frame <- function(obj){
                        sapply(obj,FUN = function(x) all(is.na(x)))
                        }
                            
isnt.finite.data.frame <- function(obj){
                        sapply(obj,FUN = function(x) all(!is.finite(x)))
                        }
           
# List of bad fitted profiles (the sleeping region is bad constrained)
badFit_DF = read.csv('BadFit.csv')
        
# Remove useless columns for the learning algorithms
DF <- originalDF                         
DF$X <- NULL  
DF$ID <- NULL    
                            
# Extracting only DF rows with well fitted sleeping regions
DF_badfit <- DF[badFit_DF$BadFit == 'True', ]
DF_notfit <- DF[badFit_DF$BadFit == 'nan', ]
nDF <- DF[badFit_DF$BadFit == 'False', ]
                               
# Creating training and test datasets
set.seed(21351)
ratioTrainTest = 5.
splitSample = sample(rep(1:ratioTrainTest, length(nDF$Gini_diff)/ratioTrainTest)) 
nDF_test <- nDF[splitSample == 1,]
nDF <- nDF[splitSample != 1,] 
# nDF <- nDF
# nDF_test <- nDF                               
                            
# Removing potential empty levels in both test and training datasets
nDF$label <- factor(nDF$label)
nDF_test$label <- factor(nDF_test$label)

In [123]:
length(nDF$label)
length(nDF_test$label)
nDF_test$label

[1] 120

[1] 30

[1] moderate mild     moderate mild     moderate moderate mild     mild    
 [9] moderate moderate mild     moderate mild     moderate severe   moderate
[17] moderate mild     mild     mild     mild     mild     mild     moderate
[25] moderate moderate mild     moderate mild     mild    
Levels: mild moderate severe

In [125]:
# Creating 10-fold training set
train <- createFolds(nDF$label, k=10)

#### Using RWeka

In [126]:
C45Fit <- train(label ~ ., method = "J48", data = nDF,
    tuneLength = 5,
    trControl = trainControl(
        method = "cv", indexOut = train))
C45Fit
C45Fit$finalModel

C4.5-like Trees 

120 samples
 25 predictor
  4 classes: 'mild', 'moderate', 'nan', 'severe' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 109, 107, 109, 109, 109, 107, ... 
Resampling results

  Accuracy   Kappa      Accuracy SD  Kappa SD 
  0.8423477  0.7294936  0.08894916   0.1435788

Tuning parameter 'C' was held constant at a value of 0.25
 

J48 pruned tree
------------------

median_HB <= 80
|   median_HB <= 60
|   |   Gini_diff <= 0.007859
|   |   |   slope <= 0.001451: mild (25.0)
|   |   |   slope > 0.001451
|   |   |   |   average_HB <= 58.771242
|   |   |   |   |   Gini_diff <= -0.001025: mild (9.0)
|   |   |   |   |   Gini_diff > -0.001025
|   |   |   |   |   |   Petrosian <= 0.565076: mild (3.0)
|   |   |   |   |   |   Petrosian > 0.565076: moderate (4.0/1.0)
|   |   |   |   average_HB > 58.771242: moderate (4.0/1.0)
|   |   Gini_diff > 0.007859: moderate (4.0)
|   median_HB > 60
|   |   SVD2 <= 0.104215: moderate (20.0/3.0)
|   |   SVD2 > 0.104215
|   |   |   Gini_t <= 0.034135: mild (8.0/1.0)
|   |   |   Gini_t > 0.034135
|   |   |   |   Hurst <= 0.794404: moderate (30.0/4.0)
|   |   |   |   Hurst > 0.794404: mild (4.0)
median_HB > 80: severe (9.0/1.0)

Number of Leaves  : 	11

Size of the tree : 	21


In [128]:
# Check resulting predictions
predictedC45 <- predict(C45Fit, nDF_test)
check <- rep(0, nrow(nDF_test))
for(i in 1:nrow(nDF_test)){
    if(c(nDF_test$label[i])-c(predictedC45[i]) == 0) check[i] <- 1
}

fracMild = sum(check[nDF_test$label == 'mild'])/length(check[nDF_test$label == 'mild'])
fracMild  
fracModerate = sum(check[nDF_test$label == 'moderate'])/length(check[nDF_test$label == 'moderate'])
fracModerate
fracSevere = sum(check[nDF_test$label == 'severe'])/length(check[nDF_test$label == 'severe'])
fracSevere


[1] 0.7333333

[1] 0.9285714

[1] 0

In [129]:
# PART (Rule-based classifier)
rulesFit <- train(label ~ ., method = "PART", data = nDF,
  tuneLength = 5,
  trControl = trainControl(
    method = "cv", indexOut = train))
rulesFit
rulesFit$finalModel

Rule-Based Classifier 

120 samples
 25 predictor
  4 classes: 'mild', 'moderate', 'nan', 'severe' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 109, 108, 107, 108, 107, 108, ... 
Resampling results

  Accuracy  Kappa      Accuracy SD  Kappa SD 
  0.823032  0.6886518  0.115787     0.2054452

Tuning parameter 'threshold' was held constant at a value of 0.25

Tuning parameter 'pruned' was held constant at a value of yes
 

PART decision list
------------------

median_HB <= 80 AND
median_HB <= 60 AND
Gini_diff <= 0.007859 AND
slope <= 0.001451: mild (25.0)

median_HB <= 80 AND
SVD2 > 0.103778 AND
Gini_t > 0.042557 AND
Hurst <= 0.794404 AND
FT_std > 0.29248 AND
Gini_t <= 0.082241: moderate (29.0/2.0)

median_HB <= 80 AND
SVD2 <= 0.103778: moderate (20.0/2.0)

median_HB <= 78 AND
Entropy_sleep > 0.042313 AND
Entropy_tot <= 0.048656: mild (14.0)

median_HB > 78: severe (9.0/1.0)

H_complexity > 386.540765: mild (6.0/1.0)

median_HB > 66.5: moderate (5.0/1.0)

H_mobility <= 0.005036 AND
Petrosian <= 0.567142: moderate (4.0)

: mild (8.0/1.0)

Number of Rules  : 	9


In [130]:
# Check resulting predictions
predictedRules <- predict(rulesFit, nDF_test)
check <- rep(0, nrow(nDF_test))
for(i in 1:nrow(nDF_test)){
    if(c(nDF_test$label[i])-c(predictedRules[i]) == 0) check[i] <- 1
}

fracMild = sum(check[nDF_test$label == 'mild'])/length(check[nDF_test$label == 'mild'])
fracMild  
fracModerate = sum(check[nDF_test$label == 'moderate'])/length(check[nDF_test$label == 'moderate'])
fracModerate
fracSevere = sum(check[nDF_test$label == 'severe'])/length(check[nDF_test$label == 'severe'])
fracSevere

[1] 0.8

[1] 0.7142857

[1] 0

In [131]:
# Linear Support Vector Machines
svmFit <- train(label ~., method = "svmLinear", data = nDF,
    tuneLength = 5,
    trControl = trainControl(
        method = "cv", indexOut = train))
svmFit
svmFit$finalModel

Support Vector Machines with Linear Kernel 

120 samples
 25 predictor
  4 classes: 'mild', 'moderate', 'nan', 'severe' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 109, 109, 109, 109, 108, 107, ... 
Resampling results

  Accuracy   Kappa     Accuracy SD  Kappa SD 
  0.7777073  0.606517  0.09086825   0.1815564

Tuning parameter 'C' was held constant at a value of 1
 

Support Vector Machine object of class "ksvm" 

SV type: C-svc  (classification) 
 parameter : cost C = 1 

Linear (vanilla) kernel function. 

Number of Support Vectors : 84 

Objective Function Value : -58.6381 -2.0195 -0.6429 -2.7926 -5.4507 -0.2351 
Training error : 0.2 

In [132]:
# Check resulting predictions
predictedSVM <- predict(svmFit, nDF_test)
check <- rep(0, nrow(nDF_test))
for(i in 1:nrow(nDF_test)){
    if(c(nDF_test$label[i])-c(predictedSVM[i]) == 0) check[i] <- 1
}

fracMild = sum(check[nDF_test$label == 'mild'])/length(check[nDF_test$label == 'mild'])
fracMild  
fracModerate = sum(check[nDF_test$label == 'moderate'])/length(check[nDF_test$label == 'moderate'])
fracModerate
fracSevere = sum(check[nDF_test$label == 'severe'])/length(check[nDF_test$label == 'severe'])
fracSevere

[1] 0.7333333

[1] 0.8571429

[1] 0

In [133]:
# Artificial Neural Network
nnetFit <- train(label ~ ., method = "nnet", data = nDF,
    tuneLength = 5,
    trControl = trainControl(
        method = "cv"
        , indexOut = train))
nnetFit
nnetFit$finalModel

# weights:  34
initial  value 172.512665 
iter  10 value 107.050216
final  value 107.034748 
converged
# weights:  94
initial  value 135.058112 
final  value 107.034748 
converged
# weights:  154
initial  value 131.551375 
final  value 107.034748 
converged
# weights:  214
initial  value 165.222137 
iter  10 value 107.034936
final  value 107.034748 
converged
# weights:  274
initial  value 181.590734 
iter  10 value 107.034750
final  value 107.034748 
converged
# weights:  34
initial  value 153.765504 
iter  10 value 107.679591
iter  20 value 96.647144
iter  30 value 85.728869
iter  40 value 83.902346
iter  50 value 83.569199
final  value 83.568959 
converged
# weights:  94
initial  value 153.339031 
iter  10 value 110.461909
iter  20 value 102.083920
iter  30 value 93.089328
iter  40 value 80.741308
iter  50 value 74.697778
iter  60 value 74.164272
iter  70 value 73.912347
iter  80 value 73.740401
iter  90 value 73.404308
iter 100 value 72.854187
final  value 72.854187 
stopped after 

Neural Network 

120 samples
 25 predictor
  4 classes: 'mild', 'moderate', 'nan', 'severe' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 109, 107, 108, 107, 108, 109, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa        Accuracy SD  Kappa SD  
  1     0e+00  0.4582168   0.00750000  0.03410079   0.04417453
  1     1e-04  0.4736930   0.02500000  0.04614126   0.07905694
  1     1e-03  0.5786830   0.23306702  0.14314397   0.27401301
  1     1e-02  0.7255778   0.50744294  0.07417313   0.14959215
  1     1e-01  0.7073959   0.47302868  0.07987981   0.15159652
  3     0e+00  0.4439311  -0.01493902  0.04332700   0.03942382
  3     1e-04  0.4796454   0.05000000  0.08800214   0.15811388
  3     1e-03  0.6641708   0.40585420  0.11758506   0.21075657
  3     1e-02  0.7697852   0.58978448  0.10406112   0.18318826
  3     1e-01  0.6767116   0.43013854  0.14244383   0.24487595
  5     0e+00  0.5630037   0.19713014  0.11572455 

a 25-3-4 network with 94 weights
inputs: DFA_dt Entropy_sleep Entropy_tot FT_std Gini_diff Gini_dt Gini_t H_complexity H_mobility Hjorth Hurst Petrosian SVD10 SVD15 SVD2 SVD20 SVD25 SVD3 SVD30 SVD5 average_HB median_HB sleepingTime slope std_DT 
output(s): .outcome 
options were - softmax modelling  decay=0.01

In [134]:
# Check resulting predictions
predictedNN <- predict(nnetFit, nDF_test)
check <- rep(0, nrow(nDF_test))
for(i in 1:nrow(nDF_test)){
    if(c(nDF_test$label[i])-c(predictedNN[i]) == 0) check[i] <- 1
}

fracMild = sum(check[nDF_test$label == 'mild'])/length(check[nDF_test$label == 'mild'])
fracMild  
fracModerate = sum(check[nDF_test$label == 'moderate'])/length(check[nDF_test$label == 'moderate'])
fracModerate
fracSevere = sum(check[nDF_test$label == 'severe'])/length(check[nDF_test$label == 'severe'])
fracSevere

[1] 0.7333333

[1] 0.7857143

[1] 0

In [135]:
# Random Forest radial
randomForestFit_SVMradial <- train(label ~ ., #method = "rf", data = nDF,
                         method = "svmRadial", data = nDF,
    tuneLength = 5,
    trControl = trainControl(
        method = "cv", indexOut = train))
randomForestFit_SVMradial
randomForestFit_SVMradial$finalModel

Support Vector Machines with Radial Basis Function Kernel 

120 samples
 25 predictor
  4 classes: 'mild', 'moderate', 'nan', 'severe' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 109, 106, 109, 109, 107, 108, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa      Accuracy SD  Kappa SD 
  0.25  0.6727123  0.4001011  0.06760393   0.1365768
  0.50  0.6889461  0.4276483  0.07994361   0.1603751
  1.00  0.7775907  0.5948342  0.12296989   0.2403515
  2.00  0.8058891  0.6535673  0.10671587   0.1977187
  4.00  0.8650833  0.7625417  0.10491299   0.1821444

Tuning parameter 'sigma' was held constant at a value of 0.03429019
Accuracy was used to select the optimal model using  the largest value.
The final values used for the model were sigma = 0.03429019 and C = 4. 

Support Vector Machine object of class "ksvm" 

SV type: C-svc  (classification) 
 parameter : cost C = 4 

Gaussian Radial Basis kernel function. 
 Hyperparameter : sigma =  0.0342901853627657 

Number of Support Vectors : 103 

Objective Function Value : -161.8154 -11.9048 -10.9345 -12.3987 -29.5544 -4.4112 
Training error : 0.091667 

In [136]:
# Check resulting predictions
predictedRF_radial <- predict(randomForestFit_SVMradial, nDF_test)
check <- rep(0, nrow(nDF_test))
for(i in 1:nrow(nDF_test)){
    if(c(nDF_test$label[i])-c(predictedRF_radial[i]) == 0) check[i] <- 1
}

fracMild = sum(check[nDF_test$label == 'mild'])/length(check[nDF_test$label == 'mild'])
fracMild  
fracModerate = sum(check[nDF_test$label == 'moderate'])/length(check[nDF_test$label == 'moderate'])
fracModerate
fracSevere = sum(check[nDF_test$label == 'severe'])/length(check[nDF_test$label == 'severe'])
fracSevere

[1] 0.6666667

[1] 0.9285714

[1] 0

In [137]:
# Random Forest linear
randomForestFit_SVMlinear <- train(label ~ ., #method = "rf", data = nDF,
                         method = "svmLinear", data = nDF,
    tuneLength = 5,
    trControl = trainControl(
        method = "cv", indexOut = train))
randomForestFit_SVMlinear
randomForestFit_SVMlinear$finalModel

Support Vector Machines with Linear Kernel 

120 samples
 25 predictor
  4 classes: 'mild', 'moderate', 'nan', 'severe' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 107, 108, 108, 108, 109, 109, ... 
Resampling results

  Accuracy   Kappa      Accuracy SD  Kappa SD 
  0.7890143  0.6299995  0.09828263   0.1788936

Tuning parameter 'C' was held constant at a value of 1
 

Support Vector Machine object of class "ksvm" 

SV type: C-svc  (classification) 
 parameter : cost C = 1 

Linear (vanilla) kernel function. 

Number of Support Vectors : 84 

Objective Function Value : -58.6381 -2.0195 -0.6429 -2.7926 -5.4507 -0.2351 
Training error : 0.2 

In [138]:
# Check resulting predictions
predictedRF_linear <- predict(randomForestFit_SVMlinear, nDF_test)
check <- rep(0, nrow(nDF_test))
for(i in 1:nrow(nDF_test)){
    if(c(nDF_test$label[i])-c(predictedRF_linear[i]) == 0) check[i] <- 1
}

fracMild = sum(check[nDF_test$label == 'mild'])/length(check[nDF_test$label == 'mild'])
fracMild  
fracModerate = sum(check[nDF_test$label == 'moderate'])/length(check[nDF_test$label == 'moderate'])
fracModerate
fracSevere = sum(check[nDF_test$label == 'severe'])/length(check[nDF_test$label == 'severe'])
fracSevere

[1] 0.7333333

[1] 0.8571429

[1] 0

In [139]:
# Random Forest standard
randomForestFit_rf <- train(label ~ ., #method = "rf", data = nDF,
                         method = "rf", data = nDF,
    tuneLength = 5,
    trControl = trainControl(
        method = "cv", indexOut = train))
randomForestFit_rf
randomForestFit_rf$finalModel

Random Forest 

120 samples
 25 predictor
  4 classes: 'mild', 'moderate', 'nan', 'severe' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 108, 108, 107, 108, 107, 109, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa      Accuracy SD  Kappa SD  
   2    0.9833333  0.9710843  0.03513642   0.06095957
   7    0.9833333  0.9710843  0.03513642   0.06095957
  13    0.9750000  0.9562695  0.04025382   0.07041946
  19    0.9833333  0.9710843  0.03513642   0.06095957
  25    0.9750000  0.9562695  0.04025382   0.07041946

Accuracy was used to select the optimal model using  the largest value.
The final value used for the model was mtry = 2. 


Call:
 randomForest(x = x, y = y, mtry = param$mtry) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 2

        OOB estimate of  error rate: 34.17%
Confusion matrix:
         mild moderate nan severe class.error
mild       37       17   0      0   0.3148148
moderate   17       37   0      1   0.3272727
nan         1        1   0      0   1.0000000
severe      0        4   0      5   0.4444444

In [140]:
# Check resulting predictions
predictedRF_rf <- predict(randomForestFit_rf, nDF_test)
check <- rep(0, nrow(nDF_test))
for(i in 1:nrow(nDF_test)){
    if(c(nDF_test$label[i])-c(predictedRF_rf[i]) == 0) check[i] <- 1
}

fracMild = sum(check[nDF_test$label == 'mild'])/length(check[nDF_test$label == 'mild'])
fracMild  
fracModerate = sum(check[nDF_test$label == 'moderate'])/length(check[nDF_test$label == 'moderate'])
fracModerate
fracSevere = sum(check[nDF_test$label == 'severe'])/length(check[nDF_test$label == 'severe'])
fracSevere

[1] 0.7333333

[1] 0.7857143

[1] 0

In [141]:
# Comparing predictions for single cases
nDF_test$best_voted <- 99
for (index in 1:length(nDF_test$Gini_t)) {
    check <- c(predict(C45Fit, nDF_test[index,]),
#                predict(svmFit, nDF_test[index,]), 
               predict(rulesFit, nDF_test[index,]), 
               predict(nnetFit, nDF_test[index,]),
                predict(randomForestFit_SVMradial, nDF_test[index,]),
                predict(randomForestFit_SVMlinear, nDF_test[index,])
#                 predict(randomForestFit_rf, nDF_test[index,])
              )
    best_voted <- names(sort(table(check),decreasing=TRUE))[1]
    nDF_test$best_voted[index] <- best_voted
}

In [142]:
diff = as.numeric(nDF_test$best_voted)-c(nDF_test$label)
(length(which(diff!=0))/ length(nDF_test$Gini_t))*100

[1] 16.66667

### Multidimensional scaling
##### (useful to separate the points in the Hjorth mobility and complexity scatter plot ???)

In [ ]:
# ...work in progress